# Data Cleaning

**Main Objective:**  
In this notebook we inspect and clean missing or duplicate values in the three main datasets:

1. **Raw CVs** (original, unstructured CV text)  
2. **Parsed Skills** (the `cv_skills` DataFrame that feeds the matcher)  
3. **Matcher Results** (output CSV from the matching algorithm)

In [ ]:
%load_ext autoreload 
%autoreload 2

import re

import polars as pl

from hiring_cv_bias.cleaning.common import (
    filter_out_candidate_ids,
    find_dropped_skill_rows,
    inspect_missing,
)
from hiring_cv_bias.cleaning.raw_cv import (
    add_length_column,
    assess_translation_completeness,
    detect_corrupted_cvs,
    detect_repetitive_cvs,
    detect_vocab_sparsity,
    filter_placeholder_tails,
    find_and_print_short_cvs,
    is_this_language,
    plot_length_histogram,
)
from hiring_cv_bias.config import (
    CANDIDATE_CVS_TRANSLATED_PATH,
    PARSED_DATA_PATH,
    REVERSE_MATCHING_PATH,
)
from hiring_cv_bias.utils import load_data, load_excel_sheets

## 1. **Raw CVs** Cleaning Steps

### 1. Schema & Load 


   - Define expected columns and data types (`CANDIDATE_ID: UInt64`, `CV_text_anon: String`, `Translated_CV: String`)  
   - Load CSV into Polars DataFrame, verify row count matches the number of uniques Candidate IDs.
   - Check for duplicate `CANDIDATE_ID` entries -> no duplicates found. 

In [ ]:
raw_cv = load_data(CANDIDATE_CVS_TRANSLATED_PATH)
print("DataFrame schema:", raw_cv.schema)
print(f"Loaded: {raw_cv.height} CVs")

################

total_rows = raw_cv.height
unique_ids = raw_cv.select(pl.col("CANDIDATE_ID")).unique().height
print(f"Unique CANDIDATE_ID: {unique_ids}")
print("\nSample rows:")
print(raw_cv.sample(5))

### 2. Missing Value Inspection 


   - Count nulls per column (`null_count()`), compute percentage of missing values and report any columns with > 0% missing

In [ ]:
missing_stats = inspect_missing(raw_cv)

### 3. Short- or Empty-CV Detection  

Under this step we will:

1. **Empty / Whitespace-Only/ Very Short Text**  
   - Filter out any records where `CV_text_anon` is below a chosen threshold (e.g. 300 characters), showing their IDs and snippets for manual review.

2. **High Repetition / Low Structure**  
   - Split each CV into non-empty lines, count `n_lines` and `unique_lines`.  
   - Compute `repetition_ratio = 1 − (unique_lines / n_lines)`.  
   - Flag and drop any CVs with `repetition_ratio` above a threshold (e.g. > 0.7).

These checks ensure we remove any CVs that are dominated by boilerplate before proceeding with further analysis.

In [ ]:
plot_length_histogram(raw_cv, text_col="CV_text_anon", bin_size=300, max_bin=3000)

Remove Empty / Whitespace-Only/ Very Short Text

In [ ]:
raw_cv = add_length_column(raw_cv, text_col="CV_text_anon", length_col="len_anon")


too_short_df = find_and_print_short_cvs(
    raw_cv,
    length_col="len_anon",
    threshold=300,
    id_col="CANDIDATE_ID",
    text_col="CV_text_anon",
)

In [ ]:
short_cvs_ids = too_short_df.select(pl.col("CANDIDATE_ID")).to_series().to_list()
raw_cv_cleaned = filter_out_candidate_ids(
    raw_cv, short_cvs_ids, df_name="CVs", description="under 300 characters"
)

Remove High Repetition / Low Structure CVs  

In [ ]:
repetitive_cvs = detect_repetitive_cvs(
    raw_cv_cleaned, text_col="CV_text_anon", max_repetition=0.5
)
print(f"Found {repetitive_cvs.height} repetitive CVs:")
print(repetitive_cvs.select(["CANDIDATE_ID", "n_lines", "repetition_ratio"]))

In [ ]:
repetitive_cvs_ids = repetitive_cvs.select(pl.col("CANDIDATE_ID")).to_series().to_list()
raw_cv_cleaned = filter_out_candidate_ids(
    raw_cv_cleaned, repetitive_cvs_ids, df_name="CVs", description="repetitive CVs"
)

### 4. Text Quality Checks

To further ensure we filter out “bad” or malformed CVs, we will perform:

1. **Token & Vocabulary Richness**  
   - Calculate the unique words / total words ratio to measure lexical variety.
   - Filter out CVs with very low number of unique words (e.g < 20) or very low ratio (e.g. < 0.2).

2. **Detect Redacted‐Placeholder Tails**
   - Flag any CV whose text ends with a long run of the same character (e.g. “XXXXXXXXXX…”), seen after manual inspection. These indicate fully redacted templates with no usable content and should be excluded.

3. **Detect Garbled/Corrupted CVs**
   - Compute the fraction of “unusual” characters (outside printable ASCII, Latin-1, or standard punctuation) in each CV. Flag and remove any CV where this fraction exceeds a small threshold (e.g. 3%), catching heavily garbled or control-code–laden documents.


These additional checks will help us catch CVs that are too short, overly repetitive, structurally invalid, or otherwise unfit for reliable parsing and downstream analysis.

**Token & Vocabulary Richness**  

In [ ]:
sparse_cvs = detect_vocab_sparsity(
    raw_cv_cleaned, text_col="CV_text_anon", min_words=30, min_ttr=0.3
)

print(f"CVs to discard based on vocabulary sparsity: {sparse_cvs.height}")
print(
    sparse_cvs.select(["CANDIDATE_ID", "total_words", "unique_words", "ttr"]).sort(
        pl.col("ttr"), descending=False
    )
)

In [ ]:
sparse_ids = sparse_cvs.select(pl.col("CANDIDATE_ID")).to_series().to_list()
raw_cv_cleaned = filter_out_candidate_ids(
    raw_cv_cleaned, sparse_ids, df_name="CVs", description="low lexical variety"
)

**Detect Redacted‐Placeholder Tails**

In [ ]:
placeholder_cvs = filter_placeholder_tails(
    raw_cv_cleaned, text_col="CV_text_anon", char="X", min_run=20
)
print(f"Found {placeholder_cvs.height} CVs with trailing X placeholders:")
print(placeholder_cvs.select(["CANDIDATE_ID"]))

In [ ]:
placeholder_ids = placeholder_cvs.select(pl.col("CANDIDATE_ID")).to_series().to_list()
raw_cv_cleaned = filter_out_candidate_ids(
    raw_cv_cleaned,
    placeholder_ids,
    df_name="CVs",
    description="trailing X placeholders",
)

**Detect Garbled/Corrupted CVs**

In [ ]:
corrupted_cvs = detect_corrupted_cvs(
    raw_cv_cleaned, text_col="CV_text_anon", max_unusual_frac=0.02
)
print(f"Corrupted CVs to discard: {corrupted_cvs.height}")
print(
    corrupted_cvs.select(["CANDIDATE_ID", "unusual_frac"]).sort(
        "unusual_frac", descending=True
    )
)

In [ ]:
corrupted_ids = corrupted_cvs.select(pl.col("CANDIDATE_ID")).to_series().to_list()
raw_cv_cleaned = filter_out_candidate_ids(
    raw_cv_cleaned, corrupted_ids, df_name="CVs", description="corrupted symbols"
)

### 5. **Translation Completeness**  


- **Filter Out Poor Translations**  
  Compute len_ratio = `len(Translated_CV) / len(CV_text_anon)` and flag any CV with <br>
  `len_ratio < 0.7` (e.g. single-character outputs or garbled text) or an empty/missing translation. <br>
  
Since these cases are very rare, we drop them outright instead of attempting a fallback or re-translation.  

In [ ]:
translation_stats_df = assess_translation_completeness(raw_cv_cleaned)
print(
    translation_stats_df.filter(pl.col("len_ratio") < 0.7)
    .select(["CANDIDATE_ID", "orig_len", "trans_len", "len_ratio"])
    .sort("len_ratio", descending=False)
)

In [ ]:
low_translation_ids = (
    translation_stats_df.filter(pl.col("len_ratio") < 0.7)
    .select("CANDIDATE_ID")
    .to_series()
    .to_list()
)
raw_cv_cleaned = filter_out_candidate_ids(
    raw_cv_cleaned,
    low_translation_ids,
    df_name="CVs",
    description="low translation completeness",
)

We ran our language‐detection check on the **Italian** `CV_text_anon` and found **155** records flagged as “not Italian.” 

**Since our downstream analysis relies exclusively on the English translations**, we’re not going to drop these files. Instead, we will now verify that the **`Translated_CV`** column truly contains English text before proceeding with bias and skill‐extraction analyses.  


In [ ]:
not_italian_df = (
    raw_cv_cleaned.with_columns(
        [
            pl.col("CV_text_anon")
            .map_elements(
                lambda s, *_: not is_this_language(s or "", "it"),
                return_dtype=pl.Boolean,
            )
            .alias("not_italian")
        ]
    )
    .select(["CANDIDATE_ID", "CV_text_anon", "not_italian"])
    .filter(pl.col("not_italian"))
)
print(f"Found {not_italian_df.height} CVs not in Italian.")
print(not_italian_df.head())

In [ ]:
print("Sample CVs not in Italian:")
print(not_italian_df.sample(1)["CV_text_anon"].item())

In [ ]:
not_english_df = (
    raw_cv_cleaned.with_columns(
        [
            pl.col("Translated_CV")
            .map_elements(
                lambda s, *_: not is_this_language(s or "", "en"),
                return_dtype=pl.Boolean,
            )
            .alias("not_english")
        ]
    )
    .select(["CANDIDATE_ID", "Translated_CV", "not_english"])
    .filter(pl.col("not_english"))
)
print(f"Found {not_english_df.height} CVs not in English.")
print(not_english_df.head())

In [ ]:
print("Sample CVs not in English:")
print(not_english_df.sample(1)["Translated_CV"].item())

We ran our check on the **Translated_CV** column and found only **16** records flagged as non-English. After a manual review, we discovered that these CVs are indeed written in English but include many Italian place names, addresses and organization titles which skew the language detector’s statistics. Since the underlying text is English and these cases are few, we will **not** drop them.

> **Note:** going forward, all regex or pattern-based quality checks should be applied **only** to the **English** `Translated_CV` field.


### 6. **Handling Empty CV Records** 

During our inspection we found many corrupted CVs. <br>
In each dedicated section, to ensure data quality and consistency across all our analyses, we will:

1. **Exclude their parsed skills**  
   Filter out any rows in our parsed skills DataFrame (`cv_skills`) corresponding to those same `CANDIDATE_ID`s.

2. **Omit their entries in the Reverse Matching results**  
   Drop records in the Reverse Matching dataset (`ReverseMatching.xlsx`) for those candidate IDs.

This cleanup step prevents entirely missing CVs from biasing our skill‐extraction and matching analyses. 

## 2. **Parsed Skills** (the `cv_skills` DataFrame that feeds the matcher)  

### 1. Schema & Load 

In [ ]:
raw_cv = load_data(CANDIDATE_CVS_TRANSLATED_PATH)
raw_skills = load_data(PARSED_DATA_PATH)
print("DataFrame schema:", raw_skills.schema)
print(f"Loaded: {raw_skills.height} Skills")

################

total_rows = raw_skills.height
unique_ids = raw_skills.select(pl.col("CANDIDATE_ID")).unique().height
print(f"Unique CANDIDATE_ID: {unique_ids}")

Exclude parsed skills of the candidates with corrupted CVs.

In [ ]:
raw_cv_ids = raw_cv.select(pl.col("CANDIDATE_ID")).to_series().to_list()
print("Number of CVs loaded ->", len(raw_cv_ids))

raw_cv_cleaned_ids = raw_cv_cleaned.select(pl.col("CANDIDATE_ID")).to_series().to_list()
print(
    "Number of CVs kept after cleaning ->",
    len(raw_cv_cleaned_ids),
)

raw_cv_deleted_ids = (
    raw_cv.filter(~pl.col("CANDIDATE_ID").is_in(raw_cv_cleaned_ids))
    .select(pl.col("CANDIDATE_ID"))
    .to_series()
    .to_list()
)
print(
    "Number of CVs deleted ->",
    len(raw_cv_deleted_ids),
)

In [ ]:
raw_skills_cleaned = filter_out_candidate_ids(
    raw_skills, raw_cv_deleted_ids, df_name="Skills", description="CVs deleted"
)

##################

# Sanity check on the new number of unique candidate IDs.

total_rows = raw_skills_cleaned.height
unique_ids = raw_skills_cleaned.select(pl.col("CANDIDATE_ID")).unique().height
print(f"Unique CANDIDATE_ID: {unique_ids}")

### 2. Missing Value Inspection 

In [ ]:
missing_skills = inspect_missing(raw_skills_cleaned)

Inspecting missing values we've found that 82 rows are missing a `Skill` value. Diving into the data, we see that these rows are all associated with the value `DRIVERSLIC` in the `Skill_Type` column. <br>
This indicates that these rows are likely placeholders for driver license information, which is however useful to our analysis. <br>
We will **not** drop these rows from the dataset.

**Remove Missing or Null Skills**

In [ ]:
invalid_rows = raw_skills_cleaned.filter(
    pl.col("Skill").is_null() & (pl.col("Skill_Type") != "DRIVERSLIC")
)
print(f"Found {invalid_rows.height} rows with missing Skill (excluding DRIVERSLIC):")
print(invalid_rows)

##################

raw_skills_cleaned = raw_skills_cleaned.filter(
    ~(pl.col("Skill").is_null() & (pl.col("Skill_Type") != "DRIVERSLIC"))
)
print(f"After dropping invalid rows, {raw_skills_cleaned.height} skill records remain.")

### 3. Find exact duplicate

In [ ]:
total_before = raw_skills_cleaned.height
unique_before = raw_skills_cleaned.unique(subset=["CANDIDATE_ID", "Skill"]).height

print(f"Total rows: {total_before}")
print(f"Unique (ID, Skill, Skill_Type): {unique_before}")

##################

if total_before != unique_before:
    raw_skills_cleaned = raw_skills_cleaned.unique(subset=["CANDIDATE_ID", "Skill"])
    print(f"Dropped {total_before - unique_before} duplicates.")
else:
    print("No duplicates found—skipping deduplication.")

### 4. Validate Skill Types

- Ensure Skill_Type only takes one of your known categories
  (`IT_Skill`, `Job_title`, `Language_Skill`, `Professional_Skill`, `DRIVERSLIC`).

In [ ]:
VALID_TYPES = [
    "IT_Skill",
    "Job_title",
    "Language_Skill",
    "Professional_Skill",
    "DRIVERSLIC",
]

all_types = raw_skills_cleaned.select("Skill_Type").unique().to_series().to_list()
invalid_types = [t for t in all_types if t not in VALID_TYPES]

##############

if invalid_types:
    print("Found invalid Skill_Type values:")
    for t in invalid_types:
        print("  -", t)
else:
    print("All Skill_Type values are valid.")

### 5. Filter Out Garbage Skill Tokens


   - Drop any `Skill` entries that are obviously malformed (e.g. length < 2 or > 100, only punctuation or digits).  
   - Optionally remove entries matching placeholder patterns (like a long run of `X`).

In [ ]:
original_skills = raw_skills_cleaned
placeholder_pattern = re.compile(r"X{5,}")

# apply the garbage‐filter again to get the “cleaned” set
cleaned_skills = original_skills.filter(
    # length 2–100
    (
        pl.col("Skill").map_elements(lambda s, *_: len(s or ""), return_dtype=pl.Int64)
        >= 2
    )
    & (
        pl.col("Skill").map_elements(lambda s, *_: len(s or ""), return_dtype=pl.Int64)
        <= 100
    )
    &
    # contains at least one letter
    pl.col("Skill").str.contains(r"[A-Za-z]", literal=False)
    &
    # not placeholder
    pl.col("Skill").map_elements(
        lambda s, *_: not bool(placeholder_pattern.search(s or "")),
        return_dtype=pl.Boolean,
    )
)

dropped_skills = original_skills.join(
    cleaned_skills.select(["CANDIDATE_ID", "Skill", "Skill_Type"]),
    on=["CANDIDATE_ID", "Skill", "Skill_Type"],
    how="anti",
)

print(f"Dropped {dropped_skills.height} skill rows:")
print(dropped_skills.head(10))

In [ ]:
dropped_skills = find_dropped_skill_rows(raw_skills)
print(f"Dropped {dropped_skills.height} skill rows:")
print(dropped_skills.head(10))

In [ ]:
dropped_skills.filter(pl.col("Skill_Type") == "DRIVERSLIC")

All the dropped_skills would be DRIVERSLIC, so we will **not** drop any rows from the skills, as previously discussed.

## 3. **Matcher Results** (output CSV from the matching algorithm) -> To be discussed

To be discussed!

In [ ]:
reversed_skills_matching_dict = load_excel_sheets(
    REVERSE_MATCHING_PATH, sheets=["Candidates"]
)
reversed_skills_matching = reversed_skills_matching_dict["Candidates"]

print("DataFrame schema:", reversed_skills_matching.schema)
print(f"Loaded: {reversed_skills_matching.height} candidates")

################

total_rows = reversed_skills_matching.height
unique_ids = reversed_skills_matching.select(pl.col("CANDIDATE_ID")).unique().height
print(f"Unique CANDIDATE_ID: {unique_ids}")

In [ ]:
missing_reversed_skills = inspect_missing(reversed_skills_matching)

## Exporting the Cleaned Data

- **Export the cleaned CVs**, the **cleaned parsed skills**, the **cleaned matcher result** to a **new** CSV file, ensuring the schema and data types are preserved.

In [ ]:
raw_cv_cleaned.write_csv(
    "../data/Adecco_Dataset_cleaned/CV_translated_cleaned.csv", separator=";"
)
raw_skills_cleaned.write_csv(
    "../data/Adecco_Dataset_cleaned/Skills_cleaned.csv", separator=";"
)